## Digital

This python notebook contains the digital software project, which has an ML model that recognizes the numbers your hands are holding up in real time

In [6]:
import cv2 as cv
import numpy as np
import time
import mediapipe as mp
import json
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
from mediapipe.tasks.python.vision import HandLandmarkerResult

BaseOptions = mp.tasks.BaseOptions
HandLandmarker = mp.tasks.vision.HandLandmarker
HandLandmarkerOptions = mp.tasks.vision.HandLandmarkerOptions
VisionRunningMode = mp.tasks.vision.RunningMode

In [7]:
detection_res = None

In [12]:
# Supporting functions

def print_result(result: HandLandmarkerResult, output_image: mp.Image, timestamp_ms: int):
    global detection_res
    detection_res = result

    if result.hand_landmarks:
        print('Hands found!: ', result)

In [9]:
base_opt = python.BaseOptions(model_asset_path='models/hand_landmarker.task')
options = vision.HandLandmarkerOptions(base_options=base_opt, num_hands=2, running_mode=mp.tasks.vision.RunningMode.LIVE_STREAM, result_callback=print_result)
detector = vision.HandLandmarker.create_from_options(options)   

I0000 00:00:1770612855.700856 11289942 gl_context.cc:407] GL version: 2.1 (2.1 Metal - 90.5), renderer: Apple M4 Pro
W0000 00:00:1770612855.704834 11289943 inference_feedback_manager.cc:121] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1770612855.708302 11289948 inference_feedback_manager.cc:121] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


In [ ]:
capture = cv.VideoCapture(0)

if not capture.isOpened():
    print("Cannot Open Camera")
    exit()

start_time = time.time()
last_timestamp_ms = 0
while True:
    ret, frame = capture.read()

    if not ret:
        print("Can't receive frame. Exiting...")
        break

   
    
    timestamp_ms = int((time.time() - start_time)* 1000)

    # if (timestamp_ms - last_timestamp_ms > 100):
    frame_rgb = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
    mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=frame_rgb)
    detector.detect_async(mp_image, timestamp_ms) 

    cv.imshow('frame',frame)

    if cv.waitKey(1) == ord('q'):
        break

capture.release()
cv.destroyAllWindows()

detector.close()